# Decision Tree Models
The Decision Tree model can be used to discover complex linear relationships between variables for either prediction, binary classification or multi-output classification. In the first example of a decision tree algorithm I have opted to examine a commonly used dataset which is freely available in Sci-kit Learn called the 'wine' dataset.

## Decision Tree Classification
The first step involves importing the relevant packages from the Python and SciKit-Learn libraries as well as the Wine dataset.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['quality'] = wine.target

# showing the first 5 entries
df.head()

Creating this table will help me decide which features to use for the Root Node position for the Decision Tree. Switching out different features in this position can have several deterministic consequences on the overall result. First I have decided to save it as a csv file which enables me to review the dataset in its entirety using spreadsheet analysis but also for preprocessing. It can be useful in case I decide to perform additional feature engineering by creating new data columns which may provide additional insight into the model.

In [ ]:
# df.to_csv(r'C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/load_wine.csv')

Looking at some of the common statistics for the dataset using the describe() function:

In [ ]:
df.describe()

In [ ]:
from sklearn import tree
from sklearn.datasets import load_wine

wine = load_wine()
X = wine.data[:, 0:13]
y = wine.target

dtc = tree.DecisionTreeClassifier(max_depth=None)
dtc = dtc.fit(X, y)

Using the plot_tree() method within the DecisionTreeClassifier model and setting the 'filled' attribute to 'True' will color the tree nodes explaining the majority class at each decision point in the tree.

One important idea to note is that the root node which appears at the very top of each decision tree can be changed in order to arrive at the lowest Gini Impurity calculation path. This is done to compare outcomes based on causation associated with different features and the goal is to reduce the Gini Impurity value to that with the lowest impurity, in this case separating the 'Good' and 'Bad' quality wines in the best possible way.

The DecisionTreeClassifier is designed to choose the best split point overall (at the root node) and does a really good job at selecting the next best split points at each decision node.

In [ ]:
tree.plot_tree(dtc, filled=True)

The visualization of this tree plot can be tidied up using matplotlib:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# important to adjust the size of the plot otherwise it ends up being too small
plt.figure(figsize=(18,12))
dtc = tree.DecisionTreeClassifier().fit(wine.data, wine.target)
plot_tree(dtc, filled=True)
plt.title("Decision Tree Trained on Red Wine Quality Features\n\n")
plt.show()

Repeating the training process but only displaying the 'max_depth' in the classifier to a value of 2 (a depth of 2 levels from the root node), or Tree depth as it's known, is a measure of how many splits a tree can make before it must arrive at a prediction. Note that in these first two examples I have applied the model to the entire dataset but later on will split it into training and test subsets.

In [ ]:
from sklearn import tree
from sklearn.datasets import load_wine

wine = load_wine()
X = wine.data[:, 0:13]
y = wine.target

# resizing plot
plt.figure(figsize=(16,7))
dtc = tree.DecisionTreeClassifier(max_depth=None)
dtc = dtc.fit(X, y)
tree.plot_tree(dtc, max_depth=2, filled=True)
plt.show()

## Using Graphviz
This method can be used as an alternative way to display the decision tree and is derived from the SciKit Learn 'tree' model set. It's worth noting I encountered some issues when installing graphviz using Conda (which took considerable time).

In [ ]:
# %conda install python-graphviz

Installing with pip may be a better option to get dot file conversion to png format to work.

In [ ]:
# %pip install graphviz

Next, in order to create a graphviz export tree diagram 'export_graphviz' must be imported first and then a filepath must be established for the 'dot' file. Then I can derive a graphviz object from this path with a few defining attributes.

In [ ]:
import graphviz
import pydotplus
from sklearn.tree import export_graphviz

#image_path='C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/wine_tree.dot'

dot_data = tree.export_graphviz(dtc, 
                        out_file=None,
                        feature_names=wine.feature_names,
                        class_names=wine.target_names,
                        rounded=True,
                        special_characters=True,
                        filled=True
                       )

graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('original_tree.png')
graph.set_size('"2,2!"')
graph.write_png('resized_tree.png')


graph = graphviz.Source(dot_data)
graph

Converting the dot file to a png file requires a stand-alone installation of graphviz, or a pip installation.

In [ ]:
#!dot -Tpng wine_tree.dot -o wine_tree.png

Next in order to try and predict unseen target data based on the labels already used, I can categorize ranked estimates for the quality of wine using a four stage model. Splitting the data first into training and test sets, fitting the DecisionTreeClassifier to the training set and using it to predict the test set to make estimates.

It's worth noting that once the predictions are made they will need to be validated using some measure of accuracy. This is determined by the Gini Impurity attribute and the degree to which all the training instances at a node belong to a particular class. A Gini value of 0.0 is completely pure meaning all the instances belong to one class only.

In [ ]:
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.25)

# Initiate a 4 step modeling pattern
# Step 1: import the model to use
from sklearn.tree import DecisionTreeClassifier

# Step 2: Make an instance of the Model and set criterion parameter to gini
dtc = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=0)

# Step 3: Train the model on the data
dtc.fit(X_train, y_train)

# Step 4: Predict labels of unseen (test) data
dtc.predict(X_test)

The total number of entries in the 'wine' dataset:

In [ ]:
df.count()

So I have 178 entries in total and 14 columns, including thirteen features and one target column. The training set is represented by 75% of the total values.

In [ ]:
print(X_train.shape)
print(y_train.shape)

And the size of the test set is 25% of the values.

In [ ]:
print(X_test.shape)
print(y_test.shape)

The action of using the predict function provides 45 estimates (25%) for the test set values based on the training data (75%). Let's see if there's any improvement if I were to change the train-test-split test size to 0.2, or 20%.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.2)
dtc = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=0)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
y_pred

## Score
The score used is from the metrics library and is specifically for multi-class classification measuring the success rate for subset accuracy. Essentially the accuracy score measures the number of times the y predictions are correct given the actual y target values, for example using a 1-d array to list the values in 'y_pred' and 'y_true', then applying the accuracy score as below. Assume there are only 4 values, the code would look a little like this:

In [ ]:
print(y_test)

Lets transpose this list of values for convenience.

In [ ]:
y_test = np.array(y_test).T
print(y_test)

In [ ]:
print(y_pred)

Comparing the two lists of actual and predicted values gives:

In [ ]:
y_test = [0,2,1,0,1,1,0,2,1,1,2,2,0,1,2,1,0,0,1,0,1,0,0,1,1,1,1,1,1,2,0,0,1,0,0,0]
y_pred = [0,2,0,0,1,0,0,2,1,1,2,2,0,0,2,1,0,0,2,0,0,0,0,1,1,1,0,1,2,2,0,0,1,0,0,0]

One way to calculate the total number of accurately predicted observations

In [ ]:
y_total = 36
y_pred_correct = 29

accuracy = (y_pred_correct / y_total) * 100
print(accuracy)

The same measurement can be achieved by applying the 'accuracy_score' function after training the decision tree model and fitting the data.

In [ ]:
from sklearn.metrics import accuracy_score

dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
# rounding to 5 decimal places
acc_dtc = round(accuracy_score(y_pred, y_test) * 100, 5)
print(acc_dtc)

So, in terms of evaluation, this model does not inspire a terrific amount of confidence in predicting unseen data correctly which is where parameter tuning will be required to improve upon this score.

## Feature Engineering
Taking a look at the correlation between various features may help to establish some ideas for engineering better optimization solutions. Then I can try fine-tuning the chosen model afterwards.

In [ ]:
correlation = df.corr()
print(correlation)

Looking at the target column entitled 'quality' the highest positive correlation is with 'alcalinity_of_ash' at 0.517859. The highest negative correlated feature is 'flavanoids' at -0.847498.

In [ ]:
# Plot the width and height of the figure
plt.figure(figsize=(10, 8))
# Plot the chosen variables, labels and color
plt.scatter(df['quality'], df['alcalinity_of_ash'], marker='o', color='purple')
plt.title('Alcalinity of Ash vs Quality\n')
plt.xlabel('Quality')
plt.ylabel('Alcalinity of Ash')
plt.show()

So the concentration of values centres around the three classes, or categories of quality which are 0, 1 and 2 with successively higher alcalinity of ash from 0 through 2. It's important to note that the range of values generally start at slightly higher minimums for each quality type.

I think this data would be more beneficial if I could tell the average 'Alcalinity of Ash' for each grade or 'Quality' of wine, so the average for 0, 1 and 2. Delving into this relationship a bit closer involves looking at the mean values for each distribution, which can be achieved by placing the different qualities in separate buckets and finding their average.

In [ ]:
mean_alcalinity = df['quality'].groupby(df['alcalinity_of_ash']).mean()
mean_alcalinity

In [ ]:
# wine quality counts for each value of 'alcalinity_of_ash'
m_a = df.groupby(df['quality'])['alcalinity_of_ash'].value_counts().unstack(fill_value=0)
df_counts = pd.DataFrame(m_a)

# print count frequency for alcalinity of ash values v quality
print(df_counts)

This provides a total of only 63 different values, or variations in 'Alcalinity of Ash'. 10 values fell under quality=0, 13 values fell under quality=1 and a further 10 values under quality=3.

In [ ]:
df['quality'].describe()

In [ ]:
df['alcalinity_of_ash'].describe()

In [ ]:
# calculating the mean values for each grade or quality gives:
mean_quality = df.groupby(['quality'])['alcalinity_of_ash'].mean()
print(mean_quality)

Therefore, now I have the 3 mean values for quality 0, 1 and 2 I will check to see if the average of all 3 corresponds to the mean value for alcalinity of ash above in the describe( ) method for the dataframe.

In [ ]:
mean = (17.037288 + 20.238028 + 21.416667) / 3
print(mean)

19.494944 is the original mean value so there is a slight discrepancy but reason I calculated the mean values for 'alacalinity of ash' for each grade or 'quality' of wine shows how it increases slightly. We can see this information displayed in the plot above.

To display the correlation matrix of features in a plot I have decided to assign the color-mapping parameter 'cmap' in the imshow() function. This helps me to understand the relationship between the predictor variables in a more visually pleasing manner.

In [ ]:
# Plot the width and height of the figure
plt.figure(figsize=(8, 8))
# Select the corr() function on the dataframe and assign it to a 'correlation' variable
correlation = df.corr()
plt.imshow(correlation, cmap = 'summer', interpolation='nearest')
plt.show()

The total number of columns or features is 14 (0 through 13). The lighter green color represents a stronger positive correlation; the darker green colors are strongly negatively correlated with each other.

A better way to show the correlation matrix is below:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.matshow(correlation, cmap='coolwarm', vmin=-1, vmax=1)

fig.colorbar(cax)
ticks = np.arange(0,len(df.columns),1)
ax.set_xticks(ticks)

ax.set_xticklabels(df.columns)
plt.xticks(rotation = 90)

ax.set_yticklabels(df.columns)
ax.set_yticks(ticks)

# print the correlation factor
for i in range(df.shape[1]):
    for j in range(14):
        text = ax.text(j, i, round(correlation.iloc[i][j],2), ha="center", va="center", color="w")
plt.show()    

In this example the darker red color represents a higher, positive correlation. The cooler blue color represents a higher, negative correlation. The lighter shades of red and blue are less correlated features, so overall I can determine the best relationships among the different features and utilize them for a better machine learning model.

Key associations occur between 'flavanoids' and 'total_phenols' (0.86) indicating a fairly high, positive, correlation between these two variables. This may contribute to a better grade or quality of wine also. Let's present this positive relationship in another visual.

In [ ]:
# code for plot showing relationship between 'flavanoids', 'total_phenols' and 'quality' in wine data
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

x = df['flavanoids']
y = df['total_phenols'] 
z = df['quality']

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, linewidths=1, alpha=.7, edgecolor='k', s = 200, c=z)
plt.title('Correlation between Flavanoids, Total Phenols and the Quality of Wine')
plt.xlabel('Flavanoids')
plt.ylabel('Total Phenols')
plt.show()

In order to describe this visualization I have chosen a 3-D graph in the form of a scatterplot. Essentially we can see there are 3 main qualities of wine labeled 0 (purple), 1 (turquoise) and 2 (yellow). Now if we look at the interaction between Flavanoids and Total Phenols it shows how the distribution of data lies between these two features.

At quality 2, I can safely say that Flavanoids and Total Phenols all reside in a fairly consistent, lower range of values which are tightly grouped together. Where the quality of wine equals 1, the range of values appear to be much larger over these two features and show some linearity. Quality equals 0 appears to have slightly higher values for both Flavanoids and Total Phenols, which are not only more tightly grouped but also more linear in their relationship.

In conclusion I would say that better quality wines should exhibit a much lower concentration of Flavanoids relative to Total Phenols.

In [ ]:
x = df['flavanoids']
y = df['total_phenols']

fig = plt.figure(figsize=(6, 6))
plt.scatter(x, y)
plt.show()

Flavanoids and Total Phenols are strongly associated with wine quality and antioxidant properties which are important constituents and appear to exhibit a positive, almost linear relationship with each other. Attributes of wine quality include color, flavor and taste. It's also important to understand the ecosystem involved in wine production which relies heavily on soil, climate (and atmospheric conditions), altitude and grape lineage.

A more detailed model would probably look into some of this data in causal analysis and may provide a better, more accurate model for classification purposes but the main point to remember is that it's important to understand the dataset's origins and the quality of information provided, otherwise the 'garbage in - garbage out' problem ensues.

## Making a Prediction
So the root node splits using the 'Proline' feature which is the most abundant amino acid in grapes. The next layer at depth 1 has two different splits, one using the 'od280/od315_of_diluted_wines' Protein concentration (111 values) and another using 'flavonoids' which represent high color content (67 values), before finally moving on to layer 2 which is the max_depth specified in this case.

I can try and classify a wine based on it's features (all 13) and the path defined by each decision boundary until we reach the end. In order to use the predict_proba() function I would have to input values for each of the 13 features in order to classify the target prediction as follows:

Calculating the probabilities of each class occurring is as follows (given a max_depth of 2 layers):

In [ ]:
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.tree import export_graphviz

wine = load_wine()
X = wine.data[:, 0:13]
y = wine.target

dtc = tree.DecisionTreeClassifier(max_depth=None)
dtc = dtc.fit(X, y)

dot_data = tree.export_graphviz(
        dtc, 
        out_file="wine.dot",
        feature_names=wine.feature_names[0:13],
        class_names=wine.target_names,
        rounded=True,
        special_characters=True,
        filled=True
    )

# choose all the features or attributes for prediction, placing them in an (m x n) matrix
dtc.predict_proba([[13.0, 1.90, 2.38, 16.8, 110, 3.0, 2.94, 0.3, 2.0, 4.9, 1.05, 3.12, 840]])

So the results are split into three possible outcomes or classes, where wine quality equals 0, 1 or 2. Each class specified or predicted will show the percentage likelihood of occurrence. My output prediction shows a 100% likelihood of wine quality falling under the '0' class.

## The Iris Dataset
Another example of this process can be repeated using another dataset from scikit-learn's databases, the Iris dataset. Note that I will not be splitting the data into training, validation or test sets for the purpose of this example which means it would be unreliable if applied to any unseen or new data.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [ ]:
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

In [ ]:
# important to adjust the size of the plot otherwise it ends up being too small
plt.figure(figsize=(16,9))
dtc = tree.DecisionTreeClassifier(max_depth=2).fit(iris.data, iris.target)
plot_tree(dtc, 
        feature_names=iris.feature_names,  
        class_names=iris.target_names,
        filled=True
    )
plt.title("Decision Tree Trained on Iris Tree Dataset\n\n")
plt.show()

In [ ]:
tree_clf.predict_proba([[5, 1.5]])

Analyzing this output leads me to the conclusion that there is a 91% probability that the type of Iris leaf image falls under the class of Iris Versicolor. Following this logically I know that the root node decision boundary specifies the petal length is <= 2.45. We know that my predictor instance has a petal length of 5 > 2.45. The next decision boundary asks if petal width is <= 1.75, so I know 1.5 is less which enables me to follow this layer to the 2nd and final layer, a leaf node with Gini=0.168, 54 samples, 49 of which fall under the class=versicolor (90.7%).

## Fine-Tuning Hyperparameters
Having already used the 'max_depth' and 'criterion' hyperparameters so far it might be prudent to explore others in order to restrict the degree of freedom and to avoid any over-fitting in the model. Observing some of the hyperparameters and attributes listed in the DecisionTreeClassifier model will enable better control to prevent lower bias and higher variance which is a side-effect of over-fitting.

This means in the existing model may memorize existing data but will not have learned from the algorithm sufficiently to be reliable when it's applied to unseen data.

First of all I'm going to try and adjust the 'criterion' parameter and see what effect this has on model accuracy:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['quality'] = wine.target

In [ ]:
X = wine.data[:, 0:13]
y = wine.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.2)
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=0, max_leaf_nodes=2)
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
predictions

In [ ]:
# print y_test and prediction values:
print(y_test)

In [ ]:
print(predictions)

In [ ]:
# rounding to 5 decimal places
acc_dtc = round(accuracy_score(predictions, y_test) * 100, 5)
print(acc_dtc)

So the score has decreased significantly changing the criterion to 'entropy' instead of 'gini'. Changing it back to 'gini' but including the 'splitter' parameter:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.2)
dtc = DecisionTreeClassifier(criterion='gini', max_depth=2, splitter='best', random_state=0, min_samples_leaf=20)
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
acc_dtc = round(accuracy_score(predictions, y_test) * 100, 5)
print(acc_dtc)

A little better, but not much. Inputting the original values gives a higher score but for some reason it's changed from 83% down to 81%.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.2)
dtc = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=0)
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
acc_dtc = round(accuracy_score(predictions, y_test) * 100, 5)
print(acc_dtc)

But if I remove the max_depth parameter altogether I get:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[wine.feature_names], df['quality'], random_state=0, test_size=0.2)
dtc = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=0)
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
acc_dtc = round(accuracy_score(predictions, y_test) * 100, 5)
print(acc_dtc)

The model's nodes are allowed to expand until all the leaves are pure and the tree is extended as far as it can go.

## Decision Tree Regression
The following example will continue with the price prediction theme for Bitcoin crypto-currency with relatively fewer features than the classification example. This time I would like to apply a Decision Tree Regression model to the entire dataset before seeking further improvement in prediction accuracy and lowering variance.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# select data for modeling
bitcoin = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

#load data into dataframe
df = pd.DataFrame(bitcoin).dropna(axis=0)
# select features and target data
X = df[["Open", "High", "Low", "Volume"]]
y = df["Close"]

## Split the Dataset into Train and Test Subsets

In [ ]:
# split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# train and fit the decision tree model
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)

In [ ]:
# predict
y_pred = tree_reg.predict(X_test)

Now trying a prediction on the working linear model (first 5 values):

In [ ]:
print(y_pred[:5])

Measuring the RMSE and r-squared score for the linear model.

In [ ]:
y_pred = tree_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, y_pred)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)
    
r2_train = r2_score(y_test, y_pred)
print(r2_train)

Next I will try dividing the dataframe into several smaller training and validation sets and perform the decision tree analysis on each. This is done using K-Fold Cross Validation and the mean validation score is used to see if there is any improvement.

## Cross Validation
This method will evaluate the Decision Tree model by splitting the training set into several smaller training and validation sets for training and evaluation separately. This is achieved by using the K-fold cross validation technique and I have split the data into 10 separate folds, cv=10 (which can be changed).

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
r2_test = r2_score(y_test, y_pred)
                         
def display_scores(scores):
    print("Scores:", scores)
    print("Mean", scores.mean())
    print("Standard Deviation", scores.std())
    print("R-Squared:", r2_test)
          
display_scores(tree_rmse_scores)

So the average of the RMSE scores has decreased relative to the R-squared value. Comparing the scores from cross validation of the DecisionTreeRegressor model to those from the LinearRegression model:

In [ ]:
# instantiate model
lin_reg = LinearRegression()

# fit model
lin_reg.fit(X_train, y_train)

lin_scores = cross_val_score(lin_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

# New Model

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target
features = iris.feature_names

Data is now loaded into X (the predictors) and y (the classifications). Define a cross-validation to check the results using decision trees:

In [ ]:
from sklearnmodel_selection import cross_val_score
from sklearn.model_selection import KFold

crossvalidation = KFold(n_splits=5, shuffle=True, random_state=1)

Define the max_depth parameter inside the iterative loop to experiment with increasing the complexity of the model

## Evaluation
With respect to the Decision Tree Classifier models improvements could be made by better exploratory data analysis, feature selection and dimensionality reduction. Also, it's worth pointing out that improvements to hyperparameter fine-tuning such as increasing any 'min' parameters and decreasing any 'max' parameters can be used to regularize the model, reducing over-fitting.

The Linear Regression model appears to reduce the Standard Error but the accuracy for the estimates is the same overall.

In [ ]:
# pickle file to go here

## Print Dependencies
Dependences are fundamental to record the computational environment.

Use watermark to print version of python, ipython, and packages, and characteristics of the computer

In [ ]:
# %pip install watermark

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,tarfile,urllib3,matplotlib,sklearn,graphviz,pydotplus

# date
print (" ")
%watermark -u -n -t -z